This notebook implements PPO and A2C in ToyCTF Environment. 

The paper 'Cyber Attack Defendse Strategies using Reinforcement Learning via Simulated Network Environment' by Bum-Sok Kim, Hye-Won Suk, Jung-Hyun Kim, Jae-Hyeok Jeong, and Min-Suk concludes after comparing QL, DQN, PPO and A2C that DQN is highly effective 

Issues:

* Kernel dying
* Notebook shutting down
* Only working for env = CyberBattleToyCtf(
    maximum_node_count=12,
    maximum_total_credentials=10,
    observation_padding=True,
    throws_on_invalid_actions=False,)

In [76]:
# %%
# !pip install stable-baselines3[extra]

# %%
from typing import cast
from cyberbattle._env.cyberbattle_env import CyberBattleEnv
from cyberbattle._env.cyberbattle_toyctf import CyberBattleToyCtf
import logging
import sys
from stable_baselines3.a2c.a2c import A2C
from stable_baselines3.ppo.ppo import PPO
import cyberbattle.agents.baseline.agent_wrapper as w
from cyberbattle._env.flatten_wrapper import FlattenObservationWrapper, FlattenActionWrapper
import os
import cyberbattle.simulation.model as model
os.environ["CUDA_LAUNCH_BLOCKING"] = '1'
retrain = ['a2c']


In [64]:
# %%

logging.basicConfig(stream=sys.stdout, level=logging.ERROR, format="%(levelname)s: %(message)s")

In [65]:
# %%
env = CyberBattleToyCtf(
    maximum_node_count=12,
    maximum_total_credentials=10,
    observation_padding=True,
    throws_on_invalid_actions=False,
)
env.environment

Environment(network=<networkx.classes.digraph.DiGraph object at 0x7fc9d2e1ffa0>, vulnerability_library={}, identifiers=Identifiers(properties=['CTFFLAG:LeakedCustomerData', 'CTFFLAG:LeakedCustomerData2', 'CTFFLAG:Readme.txt-Discover secret data', 'CTFFLAG:VMPRIVATEINFO', 'GitHub', 'MySql', 'SasUrlInCommit', 'SharepointLeakingPassword', 'Ubuntu', 'nginx/1.10.3'], ports=['GIT', 'HTTPS', 'MySQL', 'PING', 'SSH', 'SSH-key', 'su'], local_vulnerabilities=['CredScan-HomeDirectory', 'CredScanBashHistory', 'SearchEdgeHistory'], remote_vulnerabilities=['AccessDataWithSASToken', 'CredScanGitHistory', 'ListAzureResources', 'NavigateWebDirectory', 'NavigateWebDirectoryFurther', 'ScanPageContent', 'ScanPageSource', 'ScanSharepointParentDirectory']), creationTime=datetime.datetime(2024, 1, 13, 16, 52, 54, 462088), lastModified=datetime.datetime(2024, 1, 13, 16, 52, 54, 462090), version='0.1.0')

In [71]:
import sys
import logging
import gym

In [78]:
#############
# gymid = 'CyberBattleTiny-v0'
#############
gymid = "CyberBattleToyCtf-v0"
env_size = None
iteration_count = 10
training_episode_count = 20
eval_episode_count = 10
maximum_node_count = 12
maximum_total_credentials = 10
#############
# gymid = "CyberBattleChain-v0"
# env_size = 10
# iteration_count = 9000
# training_episode_count = 50
# eval_episode_count = 5
# maximum_node_count = 22
# maximum_total_credentials = 22

gym_env = gym.make(gymid)

ep = w.EnvironmentBounds.of_identifiers(
    maximum_node_count=maximum_node_count,
    maximum_total_credentials=maximum_total_credentials,
    identifiers=gym_env.identifiers
)



In [79]:
gym_env.environment

Environment(network=<networkx.classes.digraph.DiGraph object at 0x7fc9c771e130>, vulnerability_library={}, identifiers=Identifiers(properties=['CTFFLAG:LeakedCustomerData', 'CTFFLAG:LeakedCustomerData2', 'CTFFLAG:Readme.txt-Discover secret data', 'CTFFLAG:VMPRIVATEINFO', 'GitHub', 'MySql', 'SasUrlInCommit', 'SharepointLeakingPassword', 'Ubuntu', 'nginx/1.10.3'], ports=['GIT', 'HTTPS', 'MySQL', 'PING', 'SSH', 'SSH-key', 'su'], local_vulnerabilities=['CredScan-HomeDirectory', 'CredScanBashHistory', 'SearchEdgeHistory'], remote_vulnerabilities=['AccessDataWithSASToken', 'CredScanGitHistory', 'ListAzureResources', 'NavigateWebDirectory', 'NavigateWebDirectoryFurther', 'ScanPageContent', 'ScanPageSource', 'ScanSharepointParentDirectory']), creationTime=datetime.datetime(2024, 1, 13, 16, 52, 54, 462088), lastModified=datetime.datetime(2024, 1, 13, 16, 52, 54, 462090), version='0.1.0')

In [66]:
# %%
env1 = FlattenActionWrapper(gym_env)

# %%
# MultiBinary
#  'action_mask',
#  'customer_data_found',
# MultiDiscrete space
#  'nodes_privilegelevel',
#  'leaked_credentials',
#  'credential_cache_matrix'
#  'discovered_nodes_properties',

ignore_fields = [
    # DummySpace
    '_credential_cache',
    '_discovered_nodes',
    '_explored_network',
]
env1.environment

Environment(network=<networkx.classes.digraph.DiGraph object at 0x7fc9d2e1ffa0>, vulnerability_library={}, identifiers=Identifiers(properties=['CTFFLAG:LeakedCustomerData', 'CTFFLAG:LeakedCustomerData2', 'CTFFLAG:Readme.txt-Discover secret data', 'CTFFLAG:VMPRIVATEINFO', 'GitHub', 'MySql', 'SasUrlInCommit', 'SharepointLeakingPassword', 'Ubuntu', 'nginx/1.10.3'], ports=['GIT', 'HTTPS', 'MySQL', 'PING', 'SSH', 'SSH-key', 'su'], local_vulnerabilities=['CredScan-HomeDirectory', 'CredScanBashHistory', 'SearchEdgeHistory'], remote_vulnerabilities=['AccessDataWithSASToken', 'CredScanGitHistory', 'ListAzureResources', 'NavigateWebDirectory', 'NavigateWebDirectoryFurther', 'ScanPageContent', 'ScanPageSource', 'ScanSharepointParentDirectory']), creationTime=datetime.datetime(2024, 1, 13, 16, 52, 54, 462088), lastModified=datetime.datetime(2024, 1, 13, 16, 52, 54, 462090), version='0.1.0')

In [67]:
env2 = FlattenObservationWrapper(cast(CyberBattleEnv, env1), ignore_fields=ignore_fields)

Filtering out field _credential_cache
Filtering out field _discovered_nodes
Filtering out field _explored_network
// MultiBinary flattened from [12, 12, 7, 10] -> 10080
// MultiBinary flattened from [12, 3] -> 36
// MultiBinary flattened from [12, 12, 8] -> 1152
// MultiBinary already flat: 1


The cell below ran fine not in OpenAi environment but did not generate any rewards.
Would shutdown notebook in OpenAi environment. 

In [68]:
# %%
if 'a2c' in retrain:
    model_a2c = A2C("MultiInputPolicy", env2).learn(10000)
    model_a2c.save('a2c_trained_toyctf')
model = A2C("MultiInputPolicy", env2).load('a2c_trained_toyctf')
model

In [69]:
# %%
obs = env2.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env2.step(action)


In [70]:
env2.render()

,status,properties,local_attacks,remote_attacks
id,,,,
client,owned,[],[SearchEdgeHistory],[]


In [ ]:
if 'ppo' in retrain:
    model_ppo = PPO("MultiInputPolicy", env2).learn(100)
    model_ppo.save('ppo_trained_toyctf')
model = PPO("MultiInputPolicy", env2).load('ppo_trained_toyctf')


# %%
obs = env2.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env2.step(action)

env2.render()
env2.close()
